In [14]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import (
    classification_report
)
import lightgbm as lgb
import numpy as np

## Dataset Preparation

In [15]:
NUM_FOLD = 5
train_df = pd.read_parquet("train_df_slogan_ne_type_link_lp.parquet")
val_df = pd.read_parquet("val_df_slogan_ne_type_link_lp.parquet")
test_df = pd.read_parquet("test_df_slogan_ne_type_link_lp.parquet")

In [24]:
dataset = pd.read_parquet("result_df_gt_2_slogan_lp.parquet")

In [25]:
dataset

,apparato isolato,fermo parziale sito tec,None,alimentazione,signal degraded,loss of frame lof,guasto matrice,fermo parziale gsm,fermo totale gsm,fermo totale 5g,...,cardinitcard initializing,mpls tunnel rdi mpls tunnel rdi,vc unequipped ho vc unequipped,guasto controllore,vlan megaco ko,serversignalfailure odu ssf,unidentified specific problem non inviato da agent,replaceablemodulemissing eqpt,grave disservizio onuc,crdpwroff card power off xexc xexc
cluster_id2,,,,,,,,,,,,,,,,,,,,,
202301100202_8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202301100204_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202301100206_5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202301100220_1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202301100230_7,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202312200220_8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202312200222_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202312200604_3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
cluster_with_guastocavo_labels = pd.read_csv(
    "20230101-20240101_real_time_clusters_filtered_guasto_cavo.csv"
)
merge_train_df = pd.merge(
    train_df, cluster_with_guastocavo_labels, on="cluster_id2", how="left"
)
merge_train_df.set_index("cluster_id2", inplace=True)
merge_val_df = pd.merge(
    val_df, cluster_with_guastocavo_labels, on="cluster_id2", how="left"
)
merge_val_df.set_index("cluster_id2", inplace=True)
merge_test_df = pd.merge(
    test_df, cluster_with_guastocavo_labels, on="cluster_id2", how="left"
)
merge_test_df.set_index("cluster_id2", inplace=True)

In [17]:
merged_df = pd.concat([merge_train_df, merge_val_df, merge_test_df])

In [18]:
merged_df

,ad16_apparato isolato_None,adm4_apparato isolato_None,lhbts_fermo parziale sito tec_None,ad16_None_nan,ad64_None_nan,adm1_apparato isolato_None,ad64_alimentazione_None,ad16_alimentazione_None,ad64_apparato isolato_None,adm1_alimentazione_None,...,oadm_loss of signal los_eth,ad64_loss of frame_oph,None_problema laser tx_None,ptn2_signal degraded_None,rxm5_loss of signal los_gb100,adm1_loss of signal los_482,oadm1_apparato isolato_None,rmx5_loss of signal los_None,adm1_sef intermittente sef_pre,GUASTO CAVO
cluster_id2,,,,,,,,,,,,,,,,,,,,,
202303170324_7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
202306070110_2,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,True
202305180440_8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
202311300204_4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
202305180204_15,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202312070256_6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
202310260340_11,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
202306080212_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True


In [22]:
merged_colun_false = merged_df[merged_df["GUASTO CAVO"] == "False"]

In [23]:
merged_colun_false

,ad16_apparato isolato_None,adm4_apparato isolato_None,lhbts_fermo parziale sito tec_None,ad16_None_nan,ad64_None_nan,adm1_apparato isolato_None,ad64_alimentazione_None,ad16_alimentazione_None,ad64_apparato isolato_None,adm1_alimentazione_None,...,oadm_loss of signal los_eth,ad64_loss of frame_oph,None_problema laser tx_None,ptn2_signal degraded_None,rxm5_loss of signal los_gb100,adm1_loss of signal los_482,oadm1_apparato isolato_None,rmx5_loss of signal los_None,adm1_sef intermittente sef_pre,GUASTO CAVO
cluster_id2,,,,,,,,,,,,,,,,,,,,,


In [10]:

shuffled_df = merged_df.sample(frac=1, random_state=7)

fold1 = shuffled_df.iloc[: len(shuffled_df) // NUM_FOLD]
fold2 = shuffled_df.iloc[
    len(shuffled_df) // NUM_FOLD : 2 * len(shuffled_df) // NUM_FOLD
]
fold3 = shuffled_df.iloc[
    2 * len(shuffled_df) // NUM_FOLD : 3 * len(shuffled_df) // NUM_FOLD
]
fold4 = shuffled_df.iloc[
    3 * len(shuffled_df) // NUM_FOLD : 4 * len(shuffled_df) // NUM_FOLD
]
fold5 = shuffled_df.iloc[4 * len(shuffled_df) // NUM_FOLD :]

In [11]:
def print_readable_metrics(avg_metrics):
    print("Metriche Medie per Fold:\n")
    print("{:<10} {:<12} {:<12} {:<12} {:<12}".format('Label', 'Precision', 'Recall', 'F1-Score', 'Support'))
    for label, metrics in avg_metrics.items():
        if label != "accuracy":
            print("{:<10} {:<12.2f} {:<12.2f} {:<12.2f} {:<12}".format(
                label,
                metrics['precision'],
                metrics['recall'],
                metrics['f1-score'] ,
                int(metrics['support'])
            ))
    print("\nAccuracy: {:.2f}%".format(avg_metrics["accuracy"] * 100))

## Random Forest


In [13]:
results = []

for iter in range(NUM_FOLD):

    train_dfs = [
        fold for i, fold in enumerate([fold1, fold2, fold3, fold4, fold5]) if i != iter
    ]
    val_df = [fold1, fold2, fold3, fold4, fold5][iter]

    train_df = pd.concat(train_dfs)
    X_train = train_df.drop(columns=["GUASTO CAVO"])
    y_train = train_df["GUASTO CAVO"].values.ravel()

    X_val = val_df.drop(columns=["GUASTO CAVO"])
    y_val = val_df["GUASTO CAVO"]

    clf = RandomForestClassifier(n_estimators=800, random_state=42)
    clf.fit(X_train, y_train)
    y_val_pred = clf.predict(X_val)
    results.append(classification_report(y_val, y_val_pred, output_dict=True))

sum_metrics = {
    key: {"precision": 0, "recall": 0, "f1-score": 0, "support": 0}
    for key in ["False", "True"]
}
sum_metrics["accuracy"] = 0

for report in results:
    for key in ["False", "True"]:
        for metric in sum_metrics[key]:
            sum_metrics[key][metric] += report[key][metric]
    sum_metrics["accuracy"] += report["accuracy"]

avg_metrics = {
    key: {metric: total / NUM_FOLD for metric, total in sum_metrics[key].items()}
    for key in ["False", "True"]
}
avg_metrics["accuracy"] = sum_metrics["accuracy"] / NUM_FOLD

print_readable_metrics(avg_metrics)

KeyError: 'False'

## Logistic Regression


In [8]:
results = []

for iter in range(NUM_FOLD):

    train_dfs = [
        fold for i, fold in enumerate([fold1, fold2, fold3, fold4, fold5]) if i != iter
    ]
    val_df = [fold1, fold2, fold3, fold4, fold5][iter]

    train_df = pd.concat(train_dfs)
    X_train = train_df.drop(columns=["GUASTO CAVO"])
    y_train = train_df["GUASTO CAVO"].values.ravel()

    X_val = val_df.drop(columns=["GUASTO CAVO"])
    y_val = val_df["GUASTO CAVO"]

    clf = LogisticRegression(random_state=42, max_iter=200)
    clf.fit(X_train, y_train)
    y_val_pred = clf.predict(X_val)
    results.append(classification_report(y_val, y_val_pred, output_dict=True))
 
sum_metrics = {
    key: {"precision": 0, "recall": 0, "f1-score": 0, "support": 0}
    for key in ["False", "True"]
}
sum_metrics["accuracy"] = 0
   
for report in results:
    for key in ["False", "True"]:
        for metric in sum_metrics[key]:
            sum_metrics[key][metric] += report[key][metric]
    sum_metrics["accuracy"] += report["accuracy"]

avg_metrics = {
    key: {metric: total / NUM_FOLD for metric, total in sum_metrics[key].items()}
    for key in ["False", "True"]
}
avg_metrics["accuracy"] = sum_metrics["accuracy"] / NUM_FOLD

print_readable_metrics(avg_metrics)

Metriche Medie per Fold:

Label      Precision    Recall       F1-Score     Support     
False      0.59         0.20         0.29         133         
True       0.85         0.97         0.90         609         

Accuracy: 83.17%


## Support Vector Machines


In [9]:
results = []


for iter in range(NUM_FOLD):

    train_dfs = [
        fold for i, fold in enumerate([fold1, fold2, fold3, fold4, fold5]) if i != iter
    ]
    val_df = [fold1, fold2, fold3, fold4, fold5][iter]

    train_df = pd.concat(train_dfs)
    X_train = train_df.drop(columns=["GUASTO CAVO"])
    y_train = train_df["GUASTO CAVO"].values.ravel()

    X_val = val_df.drop(columns=["GUASTO CAVO"])
    y_val = val_df["GUASTO CAVO"]

    clf = SVC(kernel="linear", random_state=42)
    clf.fit(X_train, y_train)
    y_val_pred = clf.predict(X_val)
    results.append(classification_report(y_val, y_val_pred, output_dict=True))
 
sum_metrics = {
    key: {"precision": 0, "recall": 0, "f1-score": 0, "support": 0}
    for key in ["False", "True"]
}
sum_metrics["accuracy"] = 0
   
for report in results:
    for key in ["False", "True"]:
        for metric in sum_metrics[key]:
            sum_metrics[key][metric] += report[key][metric]
    sum_metrics["accuracy"] += report["accuracy"]

avg_metrics = {
    key: {metric: total / NUM_FOLD for metric, total in sum_metrics[key].items()}
    for key in ["False", "True"]
}
avg_metrics["accuracy"] = sum_metrics["accuracy"] / NUM_FOLD

print_readable_metrics(avg_metrics)

Metriche Medie per Fold:

Label      Precision    Recall       F1-Score     Support     
False      0.58         0.21         0.31         133         
True       0.85         0.97         0.90         609         

Accuracy: 83.12%


## Gradient Boosting Machines


In [13]:
results = []
# Definizione dei parametri
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "binary_logloss",
    "learning_rate": 0.05,
    "num_leaves": 31,
    "feature_fraction": 0.9,
    "force_row_wise": True,
}

num_round = 2000


for iter in range(NUM_FOLD):

    train_dfs = [
        fold for i, fold in enumerate([fold1, fold2, fold3, fold4, fold5]) if i != iter
    ]
    val_df = [fold1, fold2, fold3, fold4, fold5][iter]

    train_df = pd.concat(train_dfs)
    X_train = train_df.drop(columns=["GUASTO CAVO"])
    y_train = train_df["GUASTO CAVO"].values.ravel()

    X_val = val_df.drop(columns=["GUASTO CAVO"])
    y_val = val_df["GUASTO CAVO"]
    
    train_data = lgb.Dataset(X_train, label=y_train)
    
    bst = lgb.train(params, train_data, num_round)
    y_pred = bst.predict(X_val, num_iteration=bst.best_iteration)
    y_val_pred = np.round(y_pred)
    results.append(classification_report(y_val, y_val_pred, output_dict=True))
 
sum_metrics = {
    key: {"precision": 0, "recall": 0, "f1-score": 0, "support": 0}
    for key in ["False", "True"]
}
sum_metrics["accuracy"] = 0
   
for report in results:
    for key in ["False", "True"]:
        for metric in sum_metrics[key]:
            sum_metrics[key][metric] += report[key][metric]
    sum_metrics["accuracy"] += report["accuracy"]

avg_metrics = {
    key: {metric: total / NUM_FOLD for metric, total in sum_metrics[key].items()}
    for key in ["False", "True"]
}
avg_metrics["accuracy"] = sum_metrics["accuracy"] / NUM_FOLD

print_readable_metrics(avg_metrics)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2449, number of negative: 523
[LightGBM] [Info] Total Bins 516
[LightGBM] [Info] Number of data points in the train set: 2972, number of used features: 258
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.824024 -> initscore=1.543854
[LightGBM] [Info] Start training from score 1.543854
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2423, number of negative: 548
[LightGBM] [Info] Total Bins 540
[LightGBM] [Info] Number of data points in the train set: 2971, number of used features: 270
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.815550 -> initscore=1.486486
[LightGBM] [Info] Start training from score 1.486486
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2436, number of negative: 535
[LightGBM] [Info] Total Bins 526
[LightG